# Compute SMYLE OA skill

In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

from SMYLEutils import calendar_utils as cal
from SMYLEutils import stat_utils as stat
from SMYLEutils import mapplot_utils as maps
from SMYLEutils import colorbar_utils as cbars
from SMYLEutils import io_utils as io

In [3]:
import dask
from dask.distributed import wait
dask.__version__

'2.14.0'

## Data I/O using Dask
### Create Dask Cluster

In [3]:
# cluster.close()
# client.close()

In [4]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=3,
        memory='300GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=10GB',
        project='p93300070',
        walltime='05:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

In [5]:
cluster, client = get_ClusterClient()
cluster.scale(30)

In [6]:
cluster

## Read in CAM monthly data; Convert to Seasonal averages (DJF, MAM, JJA, SON)
- The data loading steps can take ~30 minutes
- Chosen field is returned as a dask array with leading dimensions of Y (initialization year), M (ensemble member), and L (lead season). For example, for November starts, L=1 corresponds to first DJF season.
- "time" which gives prediction verification time (centered time for a given season) is also dimensioned with (Y,L)


In [12]:
def preprocessor(ds0,nlead,field):
    """ This preprocessor is applied on an individual timeseries file basis. Edit this appropriately
    for a your analysis to speed up processing. 
    """
    ds0 = cal.time_set_mid(ds0,'time')
    d0 = ds0[field].isel(z_t=0).isel(time=slice(1, nlead)) # if not on the surface
    d0 = cal.mon_to_seas(d0)
    d0 = d0.assign_coords(L=("time", np.arange(d0.sizes["time"])+1))
    d0 = d0.swap_dims({"time": "L"})
    d0 = d0.to_dataset(name=field)
    d0 = d0.reset_coords(["time"])
    d0["time"] = d0.time.expand_dims("Y")
    return d0

In [10]:
%%time

var = 'CO3'

# SMYLE-NOV CO3 data
# process all 20 ensemble members, all start dates from 1970-2018:
field = var
datadir = '/glade/campaign/cesm/development/espwg/SMYLE/archive/'
casename = 'b.e21.BSMYLE.f09_g17.????-MM.EEE'
filetype = '.pop.h.'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
ens = 20 
nlead = 24
firstyear = 1970
lastyear  = 2018
startmonth = 11

chunk = {}
smyle11 = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor,chunks=chunk)
smyle11.nbytes/1e9 #GB

CPU times: user 25.6 s, sys: 12.6 s, total: 38.3 s
Wall time: 45.4 s


7.710969516

In [11]:
%%time
# SMYLE-NOV co3_sat_arag data
field = 'co3_sat_arag'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
smyle11_b = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor)
smyle11_b.nbytes/1e9 #GB

CPU times: user 27.3 s, sys: 12.3 s, total: 39.7 s
Wall time: 44.4 s


7.710969516

In [12]:
CO3 = smyle11.CO3.persist()
co3_sat_arag = smyle11_b.co3_sat_arag.persist()

smyle11_time = smyle11.time.load()

smyle11_pre = (CO3 / co3_sat_arag)
smyle11_pre = smyle11_pre.persist()

In [13]:
var = 'omega_arag'
smyle11_pre = smyle11_pre.to_dataset(name = var)

In [14]:
%%time

smyle11_pre = smyle11_pre[var].load()

smyle11_pre.to_netcdf(var +'.11.nc')
smyle11_time.to_netcdf(var +'.11.time.nc')

CPU times: user 10.5 s, sys: 13.1 s, total: 23.6 s
Wall time: 24.7 s


In [15]:
del smyle11, smyle11_pre, smyle11_time, 

In [16]:
%%time
var = 'CO3'

# SMYLE-FEB CO3 data
# process all 20 ensemble members, all start dates from 1970-2018:
field = var
datadir = '/glade/campaign/cesm/development/espwg/SMYLE/archive/'
casename = 'b.e21.BSMYLE.f09_g17.????-MM.EEE'
filetype = '.pop.h.'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
ens = 20 
nlead = 24
firstyear = 1970
lastyear  = 2018
startmonth = 2

chunk = {}
smyle02 = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor,chunks=chunk)
smyle02.nbytes/1e9 #GB

CPU times: user 25.9 s, sys: 11.7 s, total: 37.6 s
Wall time: 42.7 s


7.710969516

In [17]:
%%time
# SMYLE-FEB co3_sat_arag data
field = 'co3_sat_arag'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
smyle02_b = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor)
smyle02_b.nbytes/1e9 #GB

CPU times: user 28.6 s, sys: 11.8 s, total: 40.3 s
Wall time: 44.6 s


7.710969516

In [18]:
CO3 = smyle02.CO3.persist()
co3_sat_arag = smyle02_b.co3_sat_arag.persist()

smyle02_time = smyle02.time.load()

smyle02_pre = (CO3 / co3_sat_arag)
smyle02_pre = smyle02_pre.persist()

In [19]:
var = 'omega_arag'
smyle02_pre = smyle02_pre.to_dataset(name = var)

In [20]:
%%time

smyle02_pre = smyle02_pre[var].load()

smyle02_pre.to_netcdf(var +'.02.nc')
smyle02_time.to_netcdf(var +'.02.time.nc')

CPU times: user 11 s, sys: 11.6 s, total: 22.6 s
Wall time: 23.9 s


In [21]:
del smyle02, smyle02_pre, smyle02_time, 

In [15]:
%%time
var = 'CO3'

# SMYLE-MAY CO3 data
# process all 20 ensemble members, all start dates from 1970-2018:
field = var
datadir = '/glade/campaign/cesm/development/espwg/SMYLE/archive/'
casename = 'b.e21.BSMYLE.f09_g17.????-MM.EEE'
filetype = '.pop.h.'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
ens = 20 
nlead = 24
firstyear = 1970
lastyear  = 2018
startmonth = 5

chunk = {}
smyle05 = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor,chunks=chunk)
smyle05.nbytes/1e9 #GB

CPU times: user 25.4 s, sys: 11.7 s, total: 37.1 s
Wall time: 42.3 s


7.710969516

In [16]:
%%time
# SMYLE-MAY co3_sat_arag data
field = 'co3_sat_arag'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
smyle05_b = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor)
smyle05_b.nbytes/1e9 #GB

CPU times: user 25.7 s, sys: 11.5 s, total: 37.2 s
Wall time: 41.9 s


7.710969516

In [17]:
CO3 = smyle05.CO3.persist()
co3_sat_arag = smyle05_b.co3_sat_arag.persist()

smyle05_time = smyle05.time.load()

smyle05_pre = (CO3 / co3_sat_arag)
smyle05_pre = smyle05_pre.persist()

In [18]:
var = 'omega_arag'
smyle05_pre = smyle05_pre.to_dataset(name = var)

In [19]:
%%time

smyle05_pre = smyle05_pre[var].load()

smyle05_pre.to_netcdf(var +'.05.nc')
smyle05_time.to_netcdf(var +'.05.time.nc')

CPU times: user 10.2 s, sys: 13.2 s, total: 23.4 s
Wall time: 24.9 s


In [20]:
del smyle05, smyle05_pre, smyle05_time, 

In [21]:
%%time
var = 'CO3'

# SMYLE-AUG CO3 data
# process all 20 ensemble members, all start dates from 1970-2018:
field = var
datadir = '/glade/campaign/cesm/development/espwg/SMYLE/archive/'
casename = 'b.e21.BSMYLE.f09_g17.????-MM.EEE'
filetype = '.pop.h.'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
ens = 20 
nlead = 24
firstyear = 1970
lastyear  = 2018
startmonth = 8

chunk = {}
smyle08 = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor,chunks=chunk)
smyle08.nbytes/1e9 #GB

CPU times: user 28.1 s, sys: 11.7 s, total: 39.8 s
Wall time: 46 s


7.710969516

In [22]:
%%time
# SMYLE-AUG co3_sat_arag data
field = 'co3_sat_arag'
filetemplate = datadir+casename+'/ocn/proc/tseries/month_1/'+casename+filetype+field+'.*.nc'
smyle08_b = io.get_monthly_data(filetemplate,filetype,ens,nlead,field,firstyear,lastyear,startmonth,preprocessor)
smyle08_b.nbytes/1e9 #GB

CPU times: user 26 s, sys: 11.8 s, total: 37.8 s
Wall time: 42 s


7.710969516

In [23]:
CO3 = smyle08.CO3.persist()
co3_sat_arag = smyle08_b.co3_sat_arag.persist()

smyle08_time = smyle08.time.load()

smyle08_pre = (CO3 / co3_sat_arag)
smyle08_pre = smyle08_pre.persist()

In [24]:
var = 'omega_arag'
smyle08_pre = smyle08_pre.to_dataset(name = var)

In [25]:
%%time

smyle08_pre = smyle08_pre[var].load()

smyle08_pre.to_netcdf(var +'.08.nc')
smyle08_time.to_netcdf(var +'.08.time.nc')

CPU times: user 10.4 s, sys: 12.2 s, total: 22.6 s
Wall time: 23.9 s


In [26]:
del smyle08, smyle08_pre, smyle08_time, 